In [3]:
import os
import pandas as pd
from tqdm import tqdm
import pickle

In [4]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")
filename = "europarl-v7.da-en.da"
with open(filename, "r", encoding="UTF-8") as file:
    lines = file.readlines()
with open("nutids_r.pickle", "rb") as f:
    nutids_r = pickle.load(f)
with open("nutids_r_stem.pickle", "rb") as f:
    nutids_r_stem = pickle.load(f)

In [13]:
import random

step_rates = [1, 2, 3]
pre_pad_result = []
len_of_list = int(len(lines))
for i in tqdm(range(0, len_of_list//3,step_rates[0])):
    pre_pad_result.append((' '.join(lines[i:i+step_rates[0]])).split())
for i in tqdm(range(len_of_list//3, 2*len_of_list//3,step_rates[1])):
    pre_pad_result.append((' '.join(lines[i:i+step_rates[1]])).split())

random.shuffle(pre_pad_result)

100%|██████████| 328134/328134 [00:27<00:00, 11830.82it/s] 


In [14]:
scope = 10
middle = int(scope/2)
padding = int(scope/2)

In [15]:
pre_cleaned_result = [["<PAD>"]*padding + lst + ["<PAD>"]*padding for lst in tqdm(pre_pad_result)]

100%|██████████| 984400/984400 [00:39<00:00, 24998.00it/s] 


In [16]:
char = "*@;:!\"?«».,"
post_pad_result = [[word.translate(str.maketrans('', '', ''.join(char))) for word in pre_cleaned_lst] for pre_cleaned_lst in tqdm(pre_cleaned_result)]

# for i in tqdm(range(len(pre_cleaned_result))):
#     pre_cleaned_lst = pre_cleaned_result[i]
#     lst = [word.translate(str.maketrans('', '', ''.join(char))) for word in pre_cleaned_lst]
#     post_pad_result.append(lst)

100%|██████████| 984400/984400 [01:21<00:00, 12106.52it/s]


In [17]:
saved_post_pad_result = post_pad_result

In [18]:
post_pad_result = saved_post_pad_result[:50000]

In [19]:
bøjning = {0: "infinitiv", 1: "førnutidsformen"}
half_scope = int(scope/2)
big_lst = []
output_lst = []


for i in tqdm(range(len(post_pad_result))):
    current_lst = post_pad_result[i]
    for x in range(len(current_lst)):
        current_word = current_lst[x]
        try: stemmed_word = nutids_r_stem[current_word]
        except: continue
        current_dataset = current_lst[x-half_scope:x+half_scope+1]
        try: current_dataset[half_scope] = stemmed_word
        except: print(current_dataset); continue
        big_lst.append((" ".join(current_dataset)).lower())
        if nutids_r[stemmed_word][0] == current_word:
            output = 0
        elif nutids_r[stemmed_word][1] == current_word:
            output = 1
        else:
            print("ERROR")
        output_lst.append(output)


100%|██████████| 50000/50000 [00:01<00:00, 37801.45it/s]


In [20]:
df = pd.DataFrame()
df["comment_text"] = big_lst
df["label"] = output_lst

print(len(df))
df = df[:6000000]

def distribution(df):
    print(df["label"].value_counts())

distribution(df)
print(len(df))
header = ["comment_text", "label"]
df.to_csv("../Datasets/EuroparlNutidsr.csv", encoding="UTF-8", index=False, sep=";")

143961
0    92156
1    51805
Name: label, dtype: int64
143961


--- 
### Making testset for nutids-r module

In [5]:
import os
import pandas as pd
from tqdm import tqdm
import pickle

In [6]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")
filename = "europarl-v7.da-en.da"
with open(filename, "r", encoding="UTF-8") as file:
    lines = file.readlines()
with open("nutids_r_bøjninger.pickle", "rb") as f:
    nutids_r_bøjninger = pickle.load(f)
with open("nutids_r_stem.pickle", "rb") as f:
    nutids_r_stem = pickle.load(f)

In [12]:
df = pd.read_csv("../Datasets/EuroparlNutidsr_testset.csv", encoding="UTF-8", sep=";")
correct_sentences = list(df["correct"])
wrong_sentences = list(df["wrong"])
from IPython.display import clear_output

def save_df():
    df = pd.DataFrame()
    df["wrong"] = wrong_sentences
    df["correct"] = correct_sentences
    df.to_csv("../Datasets/EuroparlNutidsr_testset.csv", encoding="UTF-8", index=False, sep=";")

for line in tqdm(lines[122:1000]):
    line = line.strip("\n")
    words = line.split()
    for i, word in enumerate(words):
        print("hey")
        symbol = ""
        if word[-1] in ".,?!":
            symbol = word[-1]
            word = word[:-1]
        try: stemmed = nutids_r_stem[word]
        except: continue
        if word[-1] == "s":
            continue
        print(line)
        if nutids_r_bøjninger[stemmed][0] == word:
            bøjet_ord = nutids_r_bøjninger[stemmed][1]
        else:
            bøjet_ord = nutids_r_bøjninger[stemmed][0]
        print(f"{word} -> {bøjet_ord}")
        to_continue = int(input(f"1 for Ja, 2 for Nej."))
        clear_output()
        if to_continue == 2:
            continue
        words[i] = bøjet_ord + symbol
    new_line = " ".join(words)
    wrong_sentences.append(new_line)
    correct_sentences.append(line)
    save_df()

  2%|▏         | 16/894 [02:18<1:40:50,  6.89s/it]

hey
hey
hey
hey
hey
hey
hey
Fru formand, da jeg kan høre en smule latter fra Socialdemokraterne - jeg har fået fortalt, at brede kredse i Den Socialdemokratiske Gruppe også gerne vil have taget dette punkt af dagsordenen, fordi der ved afstemningen på Formandskonferencen ikke forelå noget votum fra arbejdsgruppen af ansvarlige kolleger i Den Socialdemokratiske Gruppe.
kan -> kunne


  2%|▏         | 16/894 [02:20<2:08:02,  8.75s/it]


ValueError: invalid literal for int() with base 10: ''

----

In [20]:
import os
import pandas as pd
from tqdm import tqdm
import pickle

In [21]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")
filename = "europarl-v7.da-en.da"
with open(filename, "r", encoding="UTF-8") as file:
    lines = file.readlines()
with open("nutids_r_bøjninger.pickle", "rb") as f:
    nutids_r_bøjninger = pickle.load(f)
with open("nutids_r_stem.pickle", "rb") as f:
    nutids_r_stem = pickle.load(f)

In [45]:
# If changed all of the dataset also needs to be changed

padding_left = 15
padding_right = 5


In [46]:
testset = []
labels = []
testset_words = []

In [47]:
df = pd.read_csv("../Datasets/EuroparlSentToPos.csv", encoding="UTF-8", sep=";")
sentences = list(df["sent"])
all_pos = list(df["pos"])
df

,sent,pos
0,Genoptagelse af sessionen,NOUN ADP NOUN
1,"Jeg erklærer Europa-Parlamentets session, der ...",PRON VERB NOUN PUNCT PRON AUX VERB NOUN DET AD...
2,"Som De kan se, indfandt det store ""år 2000-pro...",ADP PRON AUX VERB VERB DET ADJ NOUN X PRON PUN...
3,De har udtrykt ønske om en debat om dette emne...,PRON AUX VERB NOUN ADP DET NOUN ADP DET NOUN A...
4,I mellemtiden ønsker jeg - som også en del kol...,ADP NOUN VERB PRON PUNCT ADP ADV DET NOUN NOUN...
...,...,...
799995,Hiv/aids kan ikke betragtes som et isoleret pr...,NOUN AUX ADV VERB ADP DET ADJ NOUN SCONJ PRON ...
799996,Hiv/aids er tæt forbundet med malaria og tuber...,NOUN AUX ADV VERB ADP NOUN CCONJ NOUN ADV ADV ...
799997,Det er kun muligt at bekæmpe hiv/aids effektiv...,PRON AUX ADV ADJ PART VERB NOUN PUNCT SCONJ NO...
799998,Det vigtigste princip bag de europæiske politi...,DET ADJ NOUN ADP DET ADJ ADJ NOUN ADV AUX VERB...


In [48]:
def get_pos_tags(index):
    return all_pos[index].split()

for i in tqdm(range(len(sentences))):
    line = sentences[i]
    if len(str(line)) < 1 or str(line) == "nan":
        continue
    line = line.strip("\n")
    true_words = line.split()
    pos = get_pos_tags(i)
    words = ["<PAD>"]*padding_left + pos + ["<PAD>"]*padding_right
    words_with_padding = ["<PAD>"]*padding_left + true_words + ["<PAD>"]*padding_right
    for i, word in enumerate(true_words):
        try: stemmed = nutids_r_stem[word]
        except: continue
        if word[-1] == "s":
            continue
        if nutids_r_bøjninger[stemmed][0] == word:
            labels.append(1)
        else:
            labels.append(0)
        testset.append(" ".join(words[i:i+padding_left+padding_right+1]))
        current_words = words_with_padding[i:i+padding_left+padding_right+1]
        current_words[padding_left] = "[MASK]"
        if words[i+padding_left] == "VERB":
            testset_words.append(" ".join(current_words))

100%|██████████| 800000/800000 [00:18<00:00, 43805.47it/s]


In [49]:
testset_words[:10]

['<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> Jeg [MASK] Europa-Parlamentets session, der blev afbrudt',
 'session, der blev afbrudt fredag den 17. december, for genoptaget. Endnu en gang vil jeg [MASK] Dem godt nytår, og jeg',
 '<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> I mellemtiden [MASK] jeg - som også en',
 'mellemtiden ønsker jeg - som også en del kolleger har anmodet om - at vi [MASK] et minuts stilhed til minde',
 '<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> Jeg [MASK] Dem til stående at iagttage',
 '<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> Jeg opfordrer Dem til stående at [MASK] et minuts stilhed. <PAD> <PAD>',
 'være passende, hvis De, fru formand, sendte en skrivelse til Sri Lankas præsident for at [MASK] vores dybe beklagelse i forbindelse',
 'Kumar Ponnambalams død og de andre voldsomme dødsfald i Sri Lanka og for indtrængende at [MASK] præsident

In [50]:
df = pd.DataFrame(zip(testset, labels), columns=["comment_text", "label"])
df

,comment_text,label
0,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
1,PUNCT PRON AUX VERB NOUN DET ADJ NOUN ADP PUNC...,1
2,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
3,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,1
4,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
...,...,...
1799315,NOUN VERB ADJ NOUN ADP NOUN ADP PROPN CCONJ DE...,1
1799316,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
1799317,<PAD> <PAD> <PAD> <PAD> <PAD> NOUN AUX ADV VER...,0
1799318,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,1


In [51]:
df["label"].value_counts()

1    1120988
0     678332
Name: label, dtype: int64

In [52]:
comment_text = df["comment_text"].values
mistakes = [l for l in comment_text if not l.isupper()]
middle = [l.split()[padding_left] for l in comment_text]

from collections import Counter

counter = Counter(middle)
for key, value in counter.items():
    print(key, ": ", value)

VERB :  1106041
AUX :  197310
NOUN :  140494
PRON :  171940
ADJ :  68537
ADV :  39850
SCONJ :  32571
NUM :  18941
ADP :  23414
X :  209
INTJ :  2
PROPN :  11


In [53]:
df.to_csv("../Datasets/EuroparlNutidsr_trainset.csv", encoding="UTF-8", index=False, sep=";")

df2 = pd.DataFrame(zip(testset_words, labels), columns=["comment_text", "label"])
df2.to_csv("../Datasets/EuroparlNutidsrWords_trainset.csv", encoding="UTF-8", index=False, sep=";")
df2

,comment_text,label
0,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
1,"session, der blev afbrudt fredag den 17. decem...",1
2,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
3,mellemtiden ønsker jeg - som også en del kolle...,1
4,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
...,...,...
1106036,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,1
1106037,Kommissionen yder omfattende bistand til partn...,1
1106038,<PAD> <PAD> <PAD> <PAD> <PAD> Hiv/aids kan ikk...,1
1106039,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,1


In [11]:
verb_text_and_labels = [(c, l) for c, l in zip(comment_text, labels) if c.split()[padding_left] == "VERB"]
verb_text = [c for c, l in verb_text_and_labels]
verb_labels = [l for c, l in verb_text_and_labels]

df = pd.DataFrame(zip(verb_text, verb_labels), columns=["comment_text", "label"])
df.to_csv("../Datasets/EuroparlNutidsr_trainset_verbs.csv", encoding="UTF-8", index=False, sep=";")
df

,comment_text,label
0,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
1,PUNCT PRON AUX VERB NOUN DET ADJ NOUN ADP PUNC...,1
2,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
3,NOUN VERB PRON PUNCT ADP ADV DET NOUN NOUN AUX...,0
4,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
...,...,...
1106036,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
1106037,NOUN VERB ADJ NOUN ADP NOUN ADP PROPN CCONJ DE...,1
1106038,<PAD> <PAD> <PAD> <PAD> <PAD> NOUN AUX ADV VER...,0
1106039,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,1


In [12]:
df["label"].value_counts()

1    616682
0    489359
Name: label, dtype: int64

In [18]:
counts = {i: 0 for i in range(20)}
for sentence in verb_text:
    num_pads = sentence.count("<PAD>")
    counts[num_pads] += 1

from tabulate import tabulate

table = [["Number of <PAD> occurrences", "Number of sentences"]]
for i in range(20):
    table.append([i, counts[i]])

print(tabulate(table, headers="firstrow", tablefmt="github"))

|   Number of <PAD> occurrences |   Number of sentences |
|-------------------------------|-----------------------|
|                             0 |                327479 |
|                             1 |                 53576 |
|                             2 |                 60853 |
|                             3 |                 62948 |
|                             4 |                 63399 |
|                             5 |                 39898 |
|                             6 |                 41941 |
|                             7 |                 44097 |
|                             8 |                 45767 |
|                             9 |                 47626 |
|                            10 |                 47950 |
|                            11 |                 45742 |
|                            12 |                 61640 |
|                            13 |                 53294 |
|                            14 |                 96519 |
|             

Finding duplicates

In [19]:
from tqdm import tqdm

duplicates = df[df.duplicated(["comment_text"], keep=False)]
result = []
for comment in tqdm(duplicates["comment_text"].unique()):
    temp = duplicates[duplicates["comment_text"] == comment]
    if set(temp["label"]) == {0,1}:
        result.append(comment)

len(result)

  1%|          | 323/33276 [00:04<08:18, 66.13it/s]


KeyboardInterrupt: 

In [17]:
len(result)

169

In [18]:
no_guess_df = pd.DataFrame(result, columns=["comment_text"])

In [19]:
df.to_csv("../Datasets/no_guess_df.csv", encoding="UTF-8", index=False, sep=";")

---
### Reviewing nutids-r

In [30]:
import os
import pandas as pd
from tqdm import tqdm
import pickle
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")

In [36]:
df1 = pd.read_csv("nutids_r.csv", encoding="UTF-8", sep=",", header=None, names=["wrong", "correct"])
df2 = pd.read_csv("EuroparlNutidsr_testset.csv", encoding="UTF-8", sep=";")

In [37]:
df = pd.concat([df1, df2], ignore_index=True)
df

,wrong,correct
0,rigtig mange glæde sig til at ser og inviterer...,rigtig mange glæder sig til at se og invitere...
1,det er ikke altid nemt at forsvarer din opførsel,det er ikke altid nemt at forsvare din opførsel
2,mange drenge interessere sig for fodbold,mange drenge interesserer sig for fodbold
3,vil du inviterer alle dine veninder til fødsel...,vil du invitere alle dine veninder til fødsel...
4,det er svært at vurderer hvor meget bilen er værd,det er svært at vurdere hvor meget bilen er værd
...,...,...
212,"Jeg foreslå, at vi stemme om PSE-gruppens anmo...","Jeg foreslår, at vi stemmer om PSE-gruppens an..."
213,(Forslaget forkastedes) Formanden.,(Forslaget forkastedes) Formanden.
214,Stadig med hensyn til dagsordenen for onsdag h...,Stadig med hensyn til dagsordenen for onsdag h...
215,"PPE-DE-gruppen ønske, at dette punkt tages af ...","PPE-DE-gruppen ønsker, at dette punkt tages af..."
